In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from pathlib import Path

In [3]:
# Set Variables
DATA_PATH = Path("./300x225")

In [4]:
# Read the image data
train_images = np.load(DATA_PATH / "train_images.npy")
val_images = np.load(DATA_PATH / "val_images.npy")
test_images = np.load(DATA_PATH / "test_images.npy")

# Read the label data
train_labels = np.load(DATA_PATH / "train_labels.npy")
val_labels = np.load(DATA_PATH / "val_labels.npy")
test_labels = np.load(DATA_PATH / "test_labels.npy")

# Read the  metadata
md_df = pd.read_csv(DATA_PATH / "metadata.csv")

In [5]:
# Apply some horizontal flips
horizontal_flips = tf.image.flip_left_right(train_images)

In [ ]:
train_images = np.append(train_images, horizontal_flips.numpy(), axis=0)
train_labels = np.append(train_labels, train_labels)

In [ ]:
train_images.shape

(12442, 300, 225, 3)

In [ ]:
np.save(DATA_PATH / "train_images_aug.npy", train_images)
np.save(DATA_PATH / "train_labels_aug.npy", train_labels)

In [ ]:
# Set a random seed and clear back end
tf.keras.backend.clear_session()
tf.random.set_seed(1234)

# Get total number of classes
all_labels = np.concat([train_labels, val_labels, test_labels])
num_classes = len(set(all_labels))

model = tf.keras.Sequential([
    # Convolutional Layer
    tf.keras.layers.Conv2D(3, kernel_size=4, padding="same", activation="relu"),
    # Pooling Layer
    tf.keras.layers.MaxPool2D(),
    # Dropout Layer
    tf.keras.layers.Dropout(0.25),
    # Convolutional Layer
    tf.keras.layers.Conv2D(3, kernel_size=4, padding="same", activation="relu"),
    # Pooling Layer
    tf.keras.layers.MaxPool2D(),
    # Dropout Layer
    tf.keras.layers.Dropout(0.25),
    # Flattening
    tf.keras.layers.Flatten(),
    # Dense (Multiclassification Layer)
    tf.keras.layers.Dense(num_classes)
])

model.build(input_shape=(None, 300, 225, 3))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

NameError: name 'tf' is not defined

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy',
    verbose=1,
    patience=5,
    mode='max',
    restore_best_weights=True
)

In [ ]:
history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels), callbacks=[early_stopping])

Epoch 1/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 263s 670ms/step - accuracy: 0.3876 - loss: 6.4255 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 2/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 259s 666ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 3/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 273s 701ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 4/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 280s 720ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 5/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 285s 731ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 6/10
389/389 ━━━━━━━━━━━━━━━━━━━━ 312s 801ms/step - accuracy: 0.1349 - loss: 6.8807 - val_accuracy: 0.1350 - val_loss: 6.8477
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [ ]:
model.evaluate(test_images, test_labels)

72/72 ━━━━━━━━━━━━━━━━━━━━ 15s 201ms/step - accuracy: 0.2151 - loss: 7.8282


[8.067839622497559, 0.2201312929391861]